## Import

In [1]:
import pandas as pd
import numpy as np
import torch
import os
import random
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm

# for graphing
import seaborn as sns
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Seed 고정

코드를 실행한 결과가 항상 동일할 수 있도록 시드를 고정해 줍니다.

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

CFG = {
    'EPOCHS':20,
    'LEARNING_RATE':1e-5,
    'BATCH_SIZE':32,
    'SEED':41
}

seed_everything(CFG['SEED']) # Seed 고정
device = torch.device('mps')

In [14]:
train = pd.read_csv('/kaggle/input/dacon-ai-vs-human/train.csv')
test = pd.read_csv('/kaggle/input/dacon-ai-vs-human/test.csv')

train

,id,sentence1,sentence2,sentence3,sentence4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1
5,TRAIN_005,"치킨 반마리 주문 가능. 골뱅이소면과 함께 추천. 치킨은 평범, 사이드메뉴와 함께하...",둘이서 치킨 반마리 주문이 가능해서 골뱅이소면무침랑 같이 시킬 수 있었네요! 치킨맛...,"두 사람이서 반마리의 치킨을 주문할 수 있어서, 골뱅이소면무침을 함께 주문했습니다....",두 사람이서 치킨 반마리 주문이 가능해서 골뱅이소면무침과 함께 먹을 수 있었습니다....,2
6,TRAIN_006,진짜 일하시는 사람들 너무 시끄러워요 자기들 개인사얘기하는데 목소리도 크고 밥이 ...,진짜 시끄러운데... 이런거 시끄러워서 돈 아까워.,진짜 이 직장은 너무 시끄럽네요. 직원들끼리만 얘기하는게 아니라 목소리도 크고 밥먹...,"진짜 일하는 분들이 너무 시끄러웠어요. 개인적인 이야기를 할 때도 목소리가 크고, ...",1
7,TRAIN_007,"우와 정말 멋진 스터디 카페입니다. 다른 지점은 가보지 않았지만, 노트북을 많이 사...","와우, 정말 좋아요! 이 스터디 카페는 정말 최고예요. 다른 지점은 가보지 않았지만...","와우, 정말 좋은 공부 카페예요. 다른 지점은 가본 적은 없지만, 노트북을 많이 사...",우와 완전좋아요 스터디카페 신세계 ... 다른지점은 안가봤지만 노트북많이쓰는데 노트...,4
8,TRAIN_008,가시려는 분들은 이전 리뷰를 잘 읽어보고 가는 것을 추천합니다. 급하게 여기에서 상...,가시고자 하시는 분들은 앞서 쓴 후기 잘 읽어보고 가세요. 급해서 여기서 상견례했는...,이 식당을 방문하실 분들께는 앞선 리뷰들을 잘 읽어보시고 결정하시는 것을 추천드립니...,요즘은 리뷰를 믿기 힘들죠. 하지만 제 경험을 공유하고자 해요. 급하게 가서 상견례...,2
9,TRAIN_009,라운즈 앱에서 안경 구매 후 도수렌즈 구매했어요. 제가 눈이 나쁘고 렌즈 결정하기가...,라운즈 앱을 통해 안경과 도수 렌즈를 구입했습니다. 저의 시력이 좋지 않아서 렌즈 ...,라운즈 앱을 통해 안경과 도수렌즈를 구매하였습니다. 제 시력이 좋지 않아 렌즈 결정...,라운즈 앱으로 안경 구매 및 도수렌즈 선택. 눈 상태 때문에 까다롭지만 직원 친절함...,1


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [5]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
model = AutoModel.from_pretrained('skt/kogpt2-base-v2')
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2Model(
  (wte): Embedding(51200, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [ ]:
model.eval()

preds = []
with torch.no_grad():
   
    for idx in tqdm(range(len(test))):
        row = test.iloc[idx]
        best_score = float('-inf')
        best_label = 0
        
        
        random_row = train.sample(1).iloc[0]
        random_answer = random_row['label']
        random_labels = {}
        for i in range(1, 5):
            random_labels[f'sentence{i}'] = 'O' if i == random_answer else 'X'
        
        
        example_sentence = f"""
        

    

        문장1 : {random_row['sentence1']} -> {random_labels['sentence1']} \
        문장2 : {random_row['sentence2']} -> {random_labels['sentence2']} \
        문장3 : {random_row['sentence3']} -> {random_labels['sentence3']} \
        문장4 : {random_row['sentence4']} -> {random_labels['sentence4']} \


        문장 :
        """        

     
        for i in range(1, 5):
            prompt = example_sentence + " " + row[f"sentence{i}"]
            inputs = tokenizer(prompt, return_tensors="pt")
            inputs = inputs.to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                score = outputs[0][:, -1, :].max().item()

            if score > best_score:
                best_score = score
                best_label = i

        preds.append(best_label)        

 39%|███▉      | 427/1100 [00:47<01:08,  9.84it/s]

In [11]:
preds[1099]

1

In [12]:
preds = [str(pred) + '4' for pred in preds]
preds[:5]

['24', '44', '14', '44', '14']

In [13]:
submit = pd.read_csv('/kaggle/input/dacon-ai-vs-human/sample_submission.csv')
submit

,id,label
0,TEST_0000,55
1,TEST_0001,55
2,TEST_0002,55
3,TEST_0003,55
4,TEST_0004,55
...,...,...
1095,TEST_1095,55
1096,TEST_1096,55
1097,TEST_1097,55
1098,TEST_1098,55


In [ ]:
submit['label'] = preds
submit.head()

In [ ]:
submit.to_csv('./baseline_submit.csv', index=False)